In [1]:
import numpy as np 
import pandas as pd

In [2]:
import warnings
from collections import Counter
from typing import List, Tuple

import torch
from torch import nn, optim
from tqdm.auto import tqdm, trange

warnings.filterwarnings("ignore")

c:\Users\a.bromberg\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
receipts = pd.read_csv('C:/Users/a.bromberg/Downloads/receipts.csv')

## EDA

In [4]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000003 entries, 0 to 20000002
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               float64
 4   name                     object 
 5   product_sub_category_id  float64
 6   product_category_id      float64
 7   brand_id                 float64
 8   quantity                 object 
dtypes: float64(5), object(4)
memory usage: 1.3+ GB


In [5]:
receipts.head()

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
1,2018-12-07,198287,279.0,134832.0,(62448) Перекись водорода р-р наружн. 3% фл.по...,404.0,93.0,-1.0,"1,00"
2,2018-12-07,2418385,848.0,101384.0,(72183) Салициловая кислота р-р спирт 2% фл 40...,404.0,93.0,-1.0,"1,00"
3,2018-12-07,1285774,1511.0,168570.0,(197309) Пакет 28см х 50см,906.0,205.0,-1.0,"1,00"
4,2018-12-07,1810323,1501.0,168319.0,(197310) Пакет 30см х 60см,906.0,205.0,-1.0,"1,00"


Данные большие --> какие-то данные убрать?

In [6]:
receipts.isna().sum()

sale_date_date                 2
contact_id                     2
shop_id                        3
product_id                     3
name                       10741
product_sub_category_id        3
product_category_id            3
brand_id                       3
quantity                       3
dtype: int64

In [7]:
cols = ['sale_date_date', 'contact_id', 'shop_id', 'product_id', 'product_sub_category_id', 'product_category_id', 'brand_id', 'quantity']

In [8]:
receipts_num = receipts[cols]

In [9]:
receipts_num['contact_id'].value_counts()

730404         13905
665607          8677
795400          8264
995207          7455
749095          7377
               ...  
2355885            1
925767             1
956510             1
2682072            1
к: 20000000        1
Name: contact_id, Length: 1646435, dtype: int64

In [10]:
receipts_num['shop_id'].value_counts()

313.0     267255
323.0     259413
350.0     255904
316.0     251810
269.0     242599
           ...  
2081.0         1
719.0          1
1202.0         1
1688.0         1
1640.0         1
Name: shop_id, Length: 851, dtype: int64

In [11]:
receipts_num['quantity'].value_counts()
# преобразуем в num
receipts_num['quantity'] = receipts_num['quantity'].astype(str).str.replace(',', '.')
receipts_num['quantity'] = receipts_num['quantity'].astype(float)

In [12]:
receipts_num['product_id'].value_counts()
# словарь из индексов и частот встречаемости, сортированный по убыванию

129195.0    169191
168308.0    118106
158662.0     89851
110629.0     87792
137539.0     77153
             ...  
120494.0         1
224621.0         1
317844.0         1
234526.0         1
193550.0         1
Name: product_id, Length: 36549, dtype: int64

In [13]:
receipts.loc[receipts['product_id'] == 168308.0]
#необходимо проверить, как часто встречается пакет? какие ещё продукты из категории могут не учитываться в анализе?

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
0,2018-12-07,1260627,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
7,2018-12-07,1779333,1455.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
9,2018-12-07,1533558,1462.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
13,2018-12-07,1301620,1471.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
14,2018-12-07,1455298,1495.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
...,...,...,...,...,...,...,...,...,...
19990397,2018-06-13,1577194,1474.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
19990408,2018-06-13,1468160,1470.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
19990453,2018-06-13,2235049,1505.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"
19990466,2018-06-13,1664500,1482.0,168308.0,(197312) Пакет-майка 25см х 45см,906.0,205.0,-1.0,"1,00"


In [14]:
receipts.loc[receipts['product_category_id'] == 205.0, 'name'].value_counts()

(197312) Пакет-майка 25см х 45см                                        118106
(197309) Пакет 28см х 50см                                               42714
(111023) Пакет "Аптека Озерки" 28 х 35см                                 30196
(200394) Пакет-майка 40см х 70см                                         13047
(197310) Пакет 30см х 60см                                               11131
(120201) Пакет бумажный 120х240х280мм с печатью и ручками                 9157
(111024) Пакет "Аптека Озерки" 38 х 50см                                  5517
(200395) Пакет 20см х 30см                                                5279
(200396) Пакет Оптика 20см х 30см                                         4828
(72284) Пакет майка 22 х 40см                                             3880
(44975) Пакеты  п/э 25х40см с выруб ручкой N1                             3535
(57208) Пакет НД "Майка" 28х50                                            2687
(57447) Пакет "Майка" 24х44/45см                    

In [15]:
receipts.loc[receipts['product_id'] == 158662.0]
# подарочная карта, как и пакет, может быть предложена пользователю в любом случае, необходимо исключить из данных для "контекстного" анализа

,sale_date_date,contact_id,shop_id,product_id,name,product_sub_category_id,product_category_id,brand_id,quantity
12,2018-12-07,785614,859.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
48658,2018-12-07,970899,868.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
54616,2018-12-07,2737673,916.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
56843,2018-12-07,1194163,501.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
58644,2018-12-07,2711191,501.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
...,...,...,...,...,...,...,...,...,...
19825839,2018-06-10,838635,856.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
19825852,2018-06-09,895305,856.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
19969797,2018-06-12,888385,856.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"
19969805,2018-06-13,891946,856.0,158662.0,(181542) Карта Забота о Здоровье,855.0,194.0,-1.0,"1,00"


In [16]:
receipts.loc[receipts['product_category_id'] == 194.0, 'name'].value_counts()
# в категории встречаются другие товары, нельзя исключить все объекты 

(181542) Карта Забота о Здоровье                                                   89851
(197235) Хладоэлемент многократного применения МХД-1(+2до+8)                        3091
(197236) Хладоэлемент многократного применения МХД-3 (+2до+8)                         86
(204421) Хитс от Парламент стики табачные Фиолетовая пачка                             8
(188265) Шапочка-берет медицинская тип Шарлотта №50                                    8
(120140) Омрон Пикфлоуметр PFM20                                                       8
(204417) Хитс от Парламент стики табачные Бирюзовая пачка                              6
(202019) Энерджайзер макс Батарейки алкалиновые  E91/AA BP №2                          4
(204415) Хитс от Парламент стики табачные Оранжевая пачка                              4
(204423) Хитс от Парламент стики табачные Оранжевый блок                               4
(202832) Фрекен Бок Пергамент для выпечки 5+1м на гильзе бокс                          4
(179671) Пинцет со ск

In [17]:
receipts.loc[receipts['product_sub_category_id'] == 855.0, 'name'].value_counts()

(181542) Карта Забота о Здоровье                                                   89851
(197235) Хладоэлемент многократного применения МХД-1(+2до+8)                        3091
(197236) Хладоэлемент многократного применения МХД-3 (+2до+8)                         86
(204421) Хитс от Парламент стики табачные Фиолетовая пачка                             8
(188265) Шапочка-берет медицинская тип Шарлотта №50                                    8
(120140) Омрон Пикфлоуметр PFM20                                                       8
(204417) Хитс от Парламент стики табачные Бирюзовая пачка                              6
(202019) Энерджайзер макс Батарейки алкалиновые  E91/AA BP №2                          4
(204415) Хитс от Парламент стики табачные Оранжевая пачка                              4
(204423) Хитс от Парламент стики табачные Оранжевый блок                               4
(202832) Фрекен Бок Пергамент для выпечки 5+1м на гильзе бокс                          4
(179671) Пинцет со ск

Memory-based подход затруднителен, так как большой объем данных о пользователях и продуктах: затруднительно построение матриц для всех объектов. Кроме того, user-based подход столкнётся со следующими проблемами: предпочтения пользователей временные, а не постоянные, сходство пользователей обусловлено распространённостью заболеваний и подход будет не персонализированный.
Можно построить подход item-based. Проблемы: не нужно предлагать такие же товары, нужно предлагать дополняющие товары по принципу "вместе с этим берут". 
Model-based подход: также использует большой объем памяти, затруднителен на таком объеме данных

In [18]:
# наибольшее количество продаж: группируем по продукту, суммируем quantity
selled_products = receipts_num.groupby(['product_id'])['quantity'].sum().sort_values(ascending=False)

In [19]:
selled_products_100 = selled_products[selled_products.values > 100]

In [20]:
selled_products_100

product_id
39811.0     565983.45
129195.0    515576.00
158600.0    450296.00
158586.0    246185.06
95340.0     242624.05
              ...    
150585.0       101.00
144025.0       101.00
115115.0       101.00
154308.0       101.00
164923.0       100.50
Name: quantity, Length: 11223, dtype: float64

In [21]:
# сохраняем индексы, фильтруем топ самых продаваемых продуктов
ind_100_products = selled_products_100.index.tolist()

In [22]:
receipts.loc[receipts['product_id']== ind_100_products[1], 'name']
# необходимо будет удалить этот индекс из рекомендуемых продуктов

4776837     (48791) Свободная сумма НДС18% (служебный)
4776838     (48791) Свободная сумма НДС18% (служебный)
4776839     (48791) Свободная сумма НДС18% (служебный)
4776840     (48791) Свободная сумма НДС18% (служебный)
4776841     (48791) Свободная сумма НДС18% (служебный)
                               ...                    
19991340    (48791) Свободная сумма НДС18% (служебный)
19991342    (48791) Свободная сумма НДС18% (служебный)
19991344    (48791) Свободная сумма НДС18% (служебный)
19991345    (48791) Свободная сумма НДС18% (служебный)
19991346    (48791) Свободная сумма НДС18% (служебный)
Name: name, Length: 169191, dtype: object

In [23]:
ind_100_products.pop(1)

129195.0

In [24]:
receipts.loc[receipts['product_id']== ind_100_products[16], 'name'].values
# карта - объект, который можно рекомендовать каждому покупателю

array(['(181542) Карта Забота о Здоровье',
       '(181542) Карта Забота о Здоровье',
       '(181542) Карта Забота о Здоровье', ...,
       '(181542) Карта Забота о Здоровье',
       '(181542) Карта Забота о Здоровье',
       '(181542) Карта Забота о Здоровье'], dtype=object)

## Промежуточные выводы
- Необходимо убрать НДС из рекомендуемых позиций
- Карту Забота о Здоровье можно рекомендовать любому пользователю
- Рекомендовать только популярные товары будет проблематично, так как в "топе" будут товары с одним МНН, но разными торговыми названиями (т.е. одни и те же товары), а также шприцы разных объемов 
- Необходимо построить рекомендательную систему с использованием "контекста": в данном случае "контекстом" будет встречаемость товаров в одном чеке.

### Удаляем неинформативные и неактуальные позиции

In [25]:
receipts_num = receipts_num[receipts_num.product_id != 129195.0] # удаляем значения позиции "НДС"

In [26]:
receipts_num = receipts_num[receipts_num.product_category_id != 205.0] # удаляем значения из категории с пакетами

In [27]:
receipts_num

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,1.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,1.0
5,2018-12-07,2435604,66.0,37505.0,641.0,146.0,-1.0,1.0
8,2018-12-07,507339,275.0,134832.0,404.0,93.0,-1.0,1.0
12,2018-12-07,785614,859.0,158662.0,855.0,194.0,-1.0,1.0
...,...,...,...,...,...,...,...,...
19999998,2018-06-13,1924036,1485.0,71723.0,637.0,146.0,-1.0,1.0
19999999,2018-06-13,1788779,1503.0,66164.0,622.0,142.0,-1.0,1.0
20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20000001,(затронуто стр,к: 20000000,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
receipts_num = receipts_num[receipts_num.product_id != 158662.0] # удаляем значения "Карта Здоровье"

In [29]:
receipts_num

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
1,2018-12-07,198287,279.0,134832.0,404.0,93.0,-1.0,1.0
2,2018-12-07,2418385,848.0,101384.0,404.0,93.0,-1.0,1.0
5,2018-12-07,2435604,66.0,37505.0,641.0,146.0,-1.0,1.0
8,2018-12-07,507339,275.0,134832.0,404.0,93.0,-1.0,1.0
17,2018-12-08,514490,350.0,128700.0,595.0,135.0,-1.0,2.0
...,...,...,...,...,...,...,...,...
19999998,2018-06-13,1924036,1485.0,71723.0,637.0,146.0,-1.0,1.0
19999999,2018-06-13,1788779,1503.0,66164.0,622.0,142.0,-1.0,1.0
20000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20000001,(затронуто стр,к: 20000000,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
receipts_num = receipts_num.dropna()

# План реализации рекомендательной системы

1) сделать словарь с частотой встречаемости товаров
2) сформировать обучающие примеры: пары фича-таргет: чек рассматриваем отдельно как дата+contact_id, для каждого продукта в чеке выбираем его контекст (отличие от анализа текста: ближние значения не имеют смысла, контекст рассматривается как нахождение в одном чеке). Делим на батчи. 
3) определяем архитектуру сети. 
4) определимся с измерением качества.
 при создании пар разделение должно основываться на покупателе (contact_id). Ввводим случайную половину чека, на половине чека предсказываем остальную половину, затем проверяем пересечение предсказанной половины и реальной.  

## Подготовка данных

### Создадим словарь для частот встречаемости позиций 

In [31]:
receipts_int = receipts_num['product_id'].value_counts()
# словарь из индексов и частот встречаемости, сортированный по убыванию

In [32]:
ranged_list = receipts_int.index.tolist()
#создадим ранжированный список из индексов от самого часто встречающегося продукта до самого редкого
product_int = dict(zip(ranged_list, list(range(0,len(ranged_list)))))
product_int

{110629.0: 0,
 137539.0: 1,
 41096.0: 2,
 135427.0: 3,
 142296.0: 4,
 31212.0: 5,
 156491.0: 6,
 46797.0: 7,
 32834.0: 8,
 28232.0: 9,
 140162.0: 10,
 97117.0: 11,
 120300.0: 12,
 134832.0: 13,
 41890.0: 14,
 106860.0: 15,
 158600.0: 16,
 104256.0: 17,
 162780.0: 18,
 216064.0: 19,
 61115.0: 20,
 94613.0: 21,
 100907.0: 22,
 48367.0: 23,
 117333.0: 24,
 140683.0: 25,
 148601.0: 26,
 81946.0: 27,
 138005.0: 28,
 79960.0: 29,
 102788.0: 30,
 114395.0: 31,
 149071.0: 32,
 146135.0: 33,
 136478.0: 34,
 216024.0: 35,
 61540.0: 36,
 51228.0: 37,
 104501.0: 38,
 121499.0: 39,
 67609.0: 40,
 42878.0: 41,
 79724.0: 42,
 68751.0: 43,
 146664.0: 44,
 75972.0: 45,
 53779.0: 46,
 88065.0: 47,
 77238.0: 48,
 39811.0: 49,
 43027.0: 50,
 129521.0: 51,
 153978.0: 52,
 98663.0: 53,
 158586.0: 54,
 137039.0: 55,
 148022.0: 56,
 77808.0: 57,
 95340.0: 58,
 90353.0: 59,
 77064.0: 60,
 108325.0: 61,
 88371.0: 62,
 98388.0: 63,
 113441.0: 64,
 74204.0: 65,
 63026.0: 66,
 76935.0: 67,
 107409.0: 68,
 128984.0

In [33]:
receipts_num['product_id'].describe()

count    1.948356e+07
mean     9.856023e+04
std      4.424661e+04
min     -1.000000e+00
25%      6.383900e+04
50%      9.838800e+04
75%      1.326660e+05
max      3.635860e+05
Name: product_id, dtype: float64

In [34]:
# заменим значения продуктов в датасете на их частотные представления
receipts_num['product_id'] = receipts_num['product_id'].map(product_int)

In [35]:
receipts_num

,sale_date_date,contact_id,shop_id,product_id,product_sub_category_id,product_category_id,brand_id,quantity
1,2018-12-07,198287,279.0,13,404.0,93.0,-1.0,1.0
2,2018-12-07,2418385,848.0,2388,404.0,93.0,-1.0,1.0
5,2018-12-07,2435604,66.0,6960,641.0,146.0,-1.0,1.0
8,2018-12-07,507339,275.0,13,404.0,93.0,-1.0,1.0
17,2018-12-08,514490,350.0,1209,595.0,135.0,-1.0,2.0
...,...,...,...,...,...,...,...,...
19999995,2018-06-13,1601618,1499.0,111,615.0,140.0,1838.0,1.0
19999996,2018-06-13,1394104,1495.0,923,738.0,170.0,-1.0,1.0
19999997,2018-06-13,1570654,1516.0,3234,738.0,170.0,-1.0,1.0
19999998,2018-06-13,1924036,1485.0,2683,637.0,146.0,-1.0,1.0


In [36]:
#создадим словарь для декодирования
int_to_product = dict((v,k) for k,v in product_int.items())
int_to_product

{0: 110629.0,
 1: 137539.0,
 2: 41096.0,
 3: 135427.0,
 4: 142296.0,
 5: 31212.0,
 6: 156491.0,
 7: 46797.0,
 8: 32834.0,
 9: 28232.0,
 10: 140162.0,
 11: 97117.0,
 12: 120300.0,
 13: 134832.0,
 14: 41890.0,
 15: 106860.0,
 16: 158600.0,
 17: 104256.0,
 18: 162780.0,
 19: 216064.0,
 20: 61115.0,
 21: 94613.0,
 22: 100907.0,
 23: 48367.0,
 24: 117333.0,
 25: 140683.0,
 26: 148601.0,
 27: 81946.0,
 28: 138005.0,
 29: 79960.0,
 30: 102788.0,
 31: 114395.0,
 32: 149071.0,
 33: 146135.0,
 34: 136478.0,
 35: 216024.0,
 36: 61540.0,
 37: 51228.0,
 38: 104501.0,
 39: 121499.0,
 40: 67609.0,
 41: 42878.0,
 42: 79724.0,
 43: 68751.0,
 44: 146664.0,
 45: 75972.0,
 46: 53779.0,
 47: 88065.0,
 48: 77238.0,
 49: 39811.0,
 50: 43027.0,
 51: 129521.0,
 52: 153978.0,
 53: 98663.0,
 54: 158586.0,
 55: 137039.0,
 56: 148022.0,
 57: 77808.0,
 58: 95340.0,
 59: 90353.0,
 60: 77064.0,
 61: 108325.0,
 62: 88371.0,
 63: 98388.0,
 64: 113441.0,
 65: 74204.0,
 66: 63026.0,
 67: 76935.0,
 68: 107409.0,
 69: 1289

## Разобьём данные на отдельные чеки 

In [37]:
# разбиваем на чеки: группируем по дате и контакту
receipts_for_group = receipts_num[['contact_id', 'sale_date_date', 'product_id']]
receipts_grouped = receipts_for_group.groupby(['contact_id', 'sale_date_date'])
receipts_grouped.head()

,contact_id,sale_date_date,product_id
1,198287,2018-12-07,13
2,2418385,2018-12-07,2388
5,2435604,2018-12-07,6960
8,507339,2018-12-07,13
17,514490,2018-12-08,1209
...,...,...,...
19999995,1601618,2018-06-13,111
19999996,1394104,2018-06-13,923
19999997,1570654,2018-06-13,3234
19999998,1924036,2018-06-13,2683


In [38]:
receipts_for_pred =  receipts_grouped['product_id'].count().sort_values(ascending=False)

In [39]:
receipts_for_pred.values.mean()

3.2046803478910784

In [40]:
# ограничим длину чека
receipts_for_pred = receipts_for_pred[receipts_for_pred == 6]

In [41]:
receipts_for_pred.shape

(263622,)

In [42]:
receipts_for_vec = receipts_for_group.groupby(['contact_id', 'sale_date_date'])['product_id'].agg(lambda x : list(x))

In [43]:
concat_table = pd.concat([receipts_for_vec, receipts_for_pred], axis = 1, join = 'inner')

In [44]:
concat_table.columns=['products_in_check', 'product_count']

In [45]:
concat_table

,,products_in_check,product_count
contact_id,sale_date_date,,
378,2018-04-25,"[45, 45, 195, 5668, 5638, 7540]",6
396,2018-05-20,"[3035, 3610, 42, 783, 875, 1157]",6
406,2018-03-16,"[4282, 183, 7, 118, 1184, 535]",6
426,2018-05-05,"[301, 159, 25, 30, 52, 127]",6
427,2018-12-01,"[1359, 2251, 3739, 166, 808, 222]",6
...,...,...,...
2009377,2018-06-12,"[7122, 519, 609, 253, 244, 1432]",6
2009382,2018-06-13,"[6704, 3713, 2830, 1092, 2012, 11]",6
2009919,2018-06-12,"[3772, 5960, 10230, 6020, 208, 4436]",6


# Выбор метрик для оценки качества

Валидация с учётом времени не будет проведена, так как слабо выражена связь . Для валидации будет использован следующий метод: разбиваем каждый чек каждого пользователя на 2 части. Обучение будет проведено на половине чека (train). Затем для валидации будет построено предсказание на второй части чека(valid). Для оценки качества будет проведено сравнение результатов предсказания первой части чека с известными её значениями. 

# Создание тренировочной, валидирующей, тестовой выборок

In [46]:
concat_table

,,products_in_check,product_count
contact_id,sale_date_date,,
378,2018-04-25,"[45, 45, 195, 5668, 5638, 7540]",6
396,2018-05-20,"[3035, 3610, 42, 783, 875, 1157]",6
406,2018-03-16,"[4282, 183, 7, 118, 1184, 535]",6
426,2018-05-05,"[301, 159, 25, 30, 52, 127]",6
427,2018-12-01,"[1359, 2251, 3739, 166, 808, 222]",6
...,...,...,...
2009377,2018-06-12,"[7122, 519, 609, 253, 244, 1432]",6
2009382,2018-06-13,"[6704, 3713, 2830, 1092, 2012, 11]",6
2009919,2018-06-12,"[3772, 5960, 10230, 6020, 208, 4436]",6


In [47]:
prod_for_split = concat_table['products_in_check'].iloc[[0]]

In [48]:
train = []
valid = []

for row in concat_table['products_in_check']:
    '''
    splits lists into two parts:
    half&half if even number of elements
    second part 1 element greater if odd number of elements) 
    '''
    half = len(row)//2
    train.append(row[:half])
    valid.append(row[half:])


In [49]:
concat_table['train'] = train
concat_table['valid'] = valid

In [50]:
concat_table

,,products_in_check,product_count,train,valid
contact_id,sale_date_date,,,,
378,2018-04-25,"[45, 45, 195, 5668, 5638, 7540]",6,"[45, 45, 195]","[5668, 5638, 7540]"
396,2018-05-20,"[3035, 3610, 42, 783, 875, 1157]",6,"[3035, 3610, 42]","[783, 875, 1157]"
406,2018-03-16,"[4282, 183, 7, 118, 1184, 535]",6,"[4282, 183, 7]","[118, 1184, 535]"
426,2018-05-05,"[301, 159, 25, 30, 52, 127]",6,"[301, 159, 25]","[30, 52, 127]"
427,2018-12-01,"[1359, 2251, 3739, 166, 808, 222]",6,"[1359, 2251, 3739]","[166, 808, 222]"
...,...,...,...,...,...
2009377,2018-06-12,"[7122, 519, 609, 253, 244, 1432]",6,"[7122, 519, 609]","[253, 244, 1432]"
2009382,2018-06-13,"[6704, 3713, 2830, 1092, 2012, 11]",6,"[6704, 3713, 2830]","[1092, 2012, 11]"
2009919,2018-06-12,"[3772, 5960, 10230, 6020, 208, 4436]",6,"[3772, 5960, 10230]","[6020, 208, 4436]"


In [51]:
train_list = concat_table['train'].tolist()
valid_list = concat_table['valid'].tolist()

Контекстом для каждого продукта является "нахождение в одном чеке", так как расстояние между элементами в чеке не имеет значения. Для обучения взяты чеки длиной 6 позиций. В качестве "контекста" для трейна будем рассматривать нахождение в одной части чека.

# Применение метода Word2vec

In [52]:
from gensim.models import Word2Vec

In [53]:
sentences = train_list
model = Word2Vec(sentences, compute_loss=True, window=3, min_count = 1)

In [54]:
# создадим словарь с рекомендациями

Recommendations_dict = {}

for product in product_int.values():
    try:
        vector = model.wv[product]
        sims = model.wv.most_similar(vector, topn=4) 
        print (product, sims)     
        Recommendations_dict[product] = sims
        pass
    except:
        continue

0 [(0, 1.0), (174, 0.9765923023223877), (1213, 0.9741965532302856), (990, 0.9664211869239807)]
1 [(1, 1.0000001192092896), (628, 0.981172502040863), (374, 0.9801672101020813), (367, 0.9797873497009277)]
2 [(2, 1.0), (176, 0.9542481303215027), (773, 0.9365701675415039), (803, 0.927333652973175)]
3 [(3, 0.9999998211860657), (447, 0.9620288014411926), (1044, 0.9610813856124878), (532, 0.9610657691955566)]
4 [(4, 1.0), (50, 0.9723291993141174), (478, 0.9562415480613708), (438, 0.9533220529556274)]
5 [(5, 0.9999999403953552), (123, 0.9760673642158508), (117, 0.975281834602356), (280, 0.9699832201004028)]
6 [(6, 1.0), (659, 0.9301607608795166), (462, 0.9206838011741638), (957, 0.9198035597801208)]
7 [(7, 1.0), (705, 0.9549415707588196), (304, 0.9525279998779297), (654, 0.9515732526779175)]
8 [(8, 1.0), (109, 0.905939519405365), (159, 0.8972448706626892), (604, 0.8956930637359619)]
9 [(9, 1.0), (22, 0.9695948958396912), (62, 0.9667981863021851), (113, 0.9610509276390076)]
10 [(10, 1.000000119

In [55]:
Recommendations_dict 

{0: [(0, 1.0),
  (174, 0.9765923023223877),
  (1213, 0.9741965532302856),
  (990, 0.9664211869239807)],
 1: [(1, 1.0000001192092896),
  (628, 0.981172502040863),
  (374, 0.9801672101020813),
  (367, 0.9797873497009277)],
 2: [(2, 1.0),
  (176, 0.9542481303215027),
  (773, 0.9365701675415039),
  (803, 0.927333652973175)],
 3: [(3, 0.9999998211860657),
  (447, 0.9620288014411926),
  (1044, 0.9610813856124878),
  (532, 0.9610657691955566)],
 4: [(4, 1.0),
  (50, 0.9723291993141174),
  (478, 0.9562415480613708),
  (438, 0.9533220529556274)],
 5: [(5, 0.9999999403953552),
  (123, 0.9760673642158508),
  (117, 0.975281834602356),
  (280, 0.9699832201004028)],
 6: [(6, 1.0),
  (659, 0.9301607608795166),
  (462, 0.9206838011741638),
  (957, 0.9198035597801208)],
 7: [(7, 1.0),
  (705, 0.9549415707588196),
  (304, 0.9525279998779297),
  (654, 0.9515732526779175)],
 8: [(8, 1.0),
  (109, 0.905939519405365),
  (159, 0.8972448706626892),
  (604, 0.8956930637359619)],
 9: [(9, 1.0),
  (22, 0.9695948

In [56]:
new_dict = {k: Recommendations_dict[k][1:] for k in Recommendations_dict}

In [57]:
df = pd.DataFrame.from_dict(new_dict)

In [58]:
df

,0,1,2,3,4,5,6,7,8,9,...,36086,36087,36159,36299,36320,36330,36353,36369,36432,36449
0,"(174, 0.9765923023223877)","(628, 0.981172502040863)","(176, 0.9542481303215027)","(447, 0.9620288014411926)","(50, 0.9723291993141174)","(123, 0.9760673642158508)","(659, 0.9301607608795166)","(705, 0.9549415707588196)","(109, 0.905939519405365)","(22, 0.9695948958396912)",...,"(6741, 0.9426506161689758)","(6269, 0.944694459438324)","(1306, 0.799936830997467)","(8313, 0.9441280364990234)","(2913, 0.8387271761894226)","(6647, 0.9465250968933105)","(2967, 0.814082145690918)","(13243, 0.7442506551742554)","(11795, 0.8566659092903137)","(16190, 0.703266441822052)"
1,"(1213, 0.9741965532302856)","(374, 0.9801672101020813)","(773, 0.9365701675415039)","(1044, 0.9610813856124878)","(478, 0.9562415480613708)","(117, 0.975281834602356)","(462, 0.9206838011741638)","(304, 0.9525279998779297)","(159, 0.8972448706626892)","(62, 0.9667981863021851)",...,"(11643, 0.9424048066139221)","(2071, 0.9434857368469238)","(7965, 0.7994985580444336)","(4236, 0.9439805746078491)","(2440, 0.8321186900138855)","(5976, 0.9461135864257812)","(2444, 0.8123568892478943)","(8468, 0.7425428032875061)","(9764, 0.8564965128898621)","(14414, 0.7029250264167786)"
2,"(990, 0.9664211869239807)","(367, 0.9797873497009277)","(803, 0.927333652973175)","(532, 0.9610657691955566)","(438, 0.9533220529556274)","(280, 0.9699832201004028)","(957, 0.9198035597801208)","(654, 0.9515732526779175)","(604, 0.8956930637359619)","(113, 0.9610509276390076)",...,"(12461, 0.9415829181671143)","(8462, 0.9432825446128845)","(892, 0.7985925674438477)","(2718, 0.9433043599128723)","(4466, 0.8319000005722046)","(9698, 0.9458621740341187)","(2234, 0.8115940093994141)","(9511, 0.7408232688903809)","(1129, 0.8553978204727173)","(9399, 0.7012726664543152)"


In [59]:
# заменим значения колонок на id продуктов
df = df.rename(columns=int_to_product)

In [60]:
# теперь значения колонок сделаем индексами в матрице продуктов
df = df.transpose()

In [61]:
df.columns = ['first_rec', 'second_rec', 'third_rec']

In [62]:
def first_tuple_el_list(idx):
    '''
    func to:
    1. get first tuple element from each value of dataframe column
    2. decode it to product id using "int_to_product" dict
    3. append into list with the same index
    returns: list of decoded values 
    '''
    rec_list = []
    for el in df[idx]:
        el = el[0]
        rec_list.append(el)
    rec_list = (pd.Series(rec_list)).map(int_to_product).tolist()
    return rec_list

In [63]:
df['first_rec'] = first_tuple_el_list('first_rec')

In [64]:
# заменим все значения на индексы продуктов
df['second_rec'] = first_tuple_el_list('second_rec')
df['third_rec'] = first_tuple_el_list('third_rec')

# Результат подбора контекстных рекомендаций

Результат: топ-3 контекстные рекомендации для каждого id продукта представлены в таблице.

In [65]:
df

,first_rec,second_rec,third_rec
110629.0,52106.0,24714.0,106107.0
137539.0,114366.0,85223.0,127316.0
41096.0,42350.0,67520.0,91125.0
135427.0,55357.0,93251.0,137801.0
142296.0,43027.0,135500.0,145560.0
...,...,...,...
225241.0,66438.0,51152.0,85833.0
164360.0,34027.0,89366.0,62066.0
326200.0,65467.0,165246.0,164278.0
60518.0,155861.0,70950.0,73770.0


In [66]:
products = pd.read_csv("C:/Users/a.bromberg/Downloads/products.csv")

In [67]:
products.head()

,product_id,product
0,168308,(197312) Пакет-майка 25см х 45см 906
1,134832,(62448) Перекись водорода р-р наружн. 3% фл.по...
2,101384,(72183) Салициловая кислота р-р спирт 2% фл 40...
3,168570,(197309) Пакет 28см х 50см 906
4,146960,"(111023) Пакет ""Аптека Озерки"" 28 х 35см 906"


In [68]:
product_names_dict = pd.Series(products['product'].values,index=products.product_id.astype(float)).to_dict()

Подставим соответствующие значения

In [69]:
df = df.replace(product_names_dict)

К каждому id продукта (представлен индексом) выведены топ-3 рекомендуемые позиции

In [78]:
df.head(10)

,first_rec,second_rec,third_rec
110629.0,(25475) Валосердин капли оралн фл-капел 25мл N...,(97167) Корвалол капли д/приема внутрь 50мл -1,(116503) Корвалол капли д/приема внутрь 50мл -1
137539.0,(115569) Винпоцетин таб.5мг №50 639,(109257) Панангин Форте таб.п.п.о.316мг+280мг ...,(107422) Циннаризин тб 25мг N56 Renewal 639
41096.0,(87556) Хлоргексидин биглюконат р-р 0.05% фл 1...,(96724) Бриллиантовый зеленый р-р спирт 1% фл ...,(25145) Стрептоцид пор 2г пак N1 706
135427.0,(40153) Слабилен капли 0.75% 15мл фл N1 598,(101565) Эналаприл таб.20мг №28 Renewal 738,(53418) Овесол таб. №40 597
142296.0,(33764) Спазмалгон таб.№20 397,(52510) Баралгин М таб.500мг №20 397,(99310) Дротаверин тб 40мг №100 397
31212.0,(118126) Кардиомагнил таб.п.п.о. форте 150мг №...,(13804) Л-тироксин 50 Берлин Хеми табл. 50 мкг...,"(103831) Индапамид МВ Штада таб.п.о.1,5мг №30 735"
156491.0,(46161) Випросал В мазь туба 50г N1 648,(109080) Ортофен тб п/о 25мг N20 648,(67990) Диклофенак тб 50мг N20 648
46797.0,(33852) Троксевазин капс.300мг №50 736,(46045) Долгит крем 5% туба 100г N1 -1,(46057) Долгит крем 5% туба 50г N1 -1
32834.0,(102247) Вольтарен Эмульгель гель д/наружн.при...,(106255) Омепразол-Тева капсулы кишечнораствор...,(76182) Панзинорм форте 20000 таб.п.о.кишечнор...
28232.0,(102861) Тромбо АСС тб п/о кишечнораств 100мг ...,(116511) Беталок ЗОК таб.п.о.замедл.высв.25мг ...,(67560) Аторис табл. п.о. 10 мг. №90 685


Необходимо провести валидацию полученных 3-х "топовых" "контекстных" рекомендаций к каждому элементу, сравнивая значения с позициями в валидационной части чека

# Создание архитектуры нейросети для следующей ступени рекомендаций

(раздел не окончен, представлены промежуточные результаты работы)

# Генерация батчей 

В качестве контекста будем рассматривать продукты в одном чеке. Тогда в таргет для продукта по индексу заносим все элементы из списка продуктов внутри чека, кроме индекса самого продукта.

In [71]:
def get_target(products: List[int], idx: int) -> List[int]:
    """
    Get a list of products used as context of an index.

    :param products: receipt represented as a sequence of product indices
    :param idx: index of the central product that is used to make a batch
    :return: list of products in receipt except idx used as context
    """
    x = idx
    target = products[:x] + products[x+1:]
    return target

Проверим работу функции:

In [72]:
int_text = train_list[0]
idx = 2
target = get_target(int_text, idx=idx)
print("Input: ", int_text)
print(f"Index of interest: {idx}")
print("Target: ", target)

Input:  [45, 45, 195]
Index of interest: 2
Target:  [45, 45]


Создадим функцию для генерации батчей.  

In [73]:
def get_batches(
    products: List[int], batch_size: 5) -> Tuple[List[int], List[int]]:
    """"
    get batches for each index with target func
    """

    for i in range(0, len(products) // batch_size * batch_size, batch_size):
        x, y = [], []
        batch = products[i : i + batch_size]
        
        for j in range(len(batch)):
            batch_x = batch[j]
            batch_y = get_target(products, i + j)
            y.extend(batch_y)
            x.extend([batch_x] * len(batch_y))
        
        yield x, y

Проверим работу функции

In [74]:
int_list = train_list[0]
batch_gen = get_batches(int_list, batch_size=2)
x, y = next(batch_gen)

print(f"x: {x}")
print(f"y: {y}\n")



x: [45, 45, 45, 45]
y: [45, 195, 45, 195]



Создадим функцию для поиска косинусного расстояния 

In [75]:
import torch.nn.functional as F

def cosine_similarity(tensor1, tensor2):
    
    """_summary_ 
computes cosine similarity between two tensors
may be used for validation 
    Returns:
        cosine similarity value  """
    
    
    cosine_similarity_value = F.cosine_similarity(tensor1, tensor2, dim=0, eps=1e-6) 
    return cosine_similarity_value

Создадим архитектуру сети

In [76]:
class Recommend(nn.Module):
    'neural network for classification'
    def __init__(self, vocab_size = len(train_list), hidden_size = 512):
        super().__init__()
        self.flatten = nn.Flatten()
        self.hidden = nn.Linear(vocab_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x: torch.Tensor):
        out = self.hidden(x)
        out = self.linear(out)
        
        return out        

In [77]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print_every = 1000
steps = 0
n_epochs = 5
batch_size = 2000
hidden_dim = 128

model = Recommend(len(train_list), hidden_dim)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

for e in trange(n_epochs, leave=True, desc="Epoch number"):
    pbar = tqdm(
        get_batches(train_list, batch_size),
        leave=False,
        desc="Batch number",
        total=len(train_list) // batch_size,
    )

    # get input and target batches
    for inputs, targets in pbar:
        steps += 1
        inputs, targets = torch.FloatTensor(inputs), torch.FloatTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)

        log_ps = model(inputs)
        loss = criterion(log_ps, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


Epoch number:   0%|          | 0/5 [00:22<?, ?it/s]


MemoryError: 

In [ ]:
vocab_size = len(train_list)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = Recommend(vocab_size)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)
model = Recommend().to(device)
print(model)

In [ ]:
# predict: 
# input: [id1], [id2]

# output: линейный выход для каждого input, тк он имеет размерность, равную общему количеству товаров
# id1: [значение продукта 1, значение продукта 2, ... , значение продукта N] -> argmax [torch.max] -> top N max
# id2: [значение продукта 1, значение продукта 2, ... , значение продукта N] -> argmax [torch.max] -> top N max
# ..................
# idN: [значение продукта 1, значение продукта 2, ..., продукта N] -> argmax [torch.max] -> top N max